# Object Detection Video Application

Develop an object detection video application of the SST Foyer using Yolo architecture.

In [1]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk

In [2]:
# Global variables
net, output_layers = None, None
classes = []
video_path = None
video_label = None
cap = None

In [3]:
# Tkinter window
root = None

In [4]:
def load_yolo():
    global net, output_layers
    try:
        weights_file = '../cfg/yolov3.weights'
        config_file = '../cfg/yolov3.cfg'

        if not os.path.exists(weights_file) or not os.path.exists(config_file):
            raise FileNotFoundError("YOLO model files not found.")

        net = cv2.dnn.readNet(weights_file, config_file)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

        return net, output_layers
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return None, None

def load_coco_names():
    global classes
    try:
        coco_names_file = '../cfg/coco.names'
        if not os.path.exists(coco_names_file):
            raise FileNotFoundError(f"File not found: {coco_names_file}")

        with open(coco_names_file, "r") as f:
            classes = [line.strip() for line in f.readlines()]

        return classes
    except Exception as e:
        print(f"Error loading COCO class labels: {e}")
        return []

In [5]:
def detect_objects(frame):
    global net, output_layers, classes
    try:
        height, width, _ = frame.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Showing information on the screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-maximum suppression
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Draw bounding boxes and labels
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                confidence_percentage = confidences[i] * 100
                label = f"{classes[class_ids[i]]}: {confidence_percentage:.2f}%"
                color = np.random.uniform(0, 255, 3)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return frame

    except Exception as e:
        print(f"Error detecting objects: {e}")
        return frame


In [6]:
def play_video():
    global video_path, video_label, cap
    if video_path is not None:
        if isinstance(video_path, str):
            # If video_path is a string, it's a file path
            cap = cv2.VideoCapture(video_path)
        else:
            # If video_path is not a string, assume it's the camera index
            cap = cv2.VideoCapture(int(video_path))

        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = detect_objects(frame)
                frame = Image.fromarray(frame)
                video_image = ImageTk.PhotoImage(frame)
                video_label.configure(image=video_image)
                video_label.image = video_image
                video_label.update()
            else:
                break
        cap.release()
    else:
        messagebox.showerror("Error", "No video selected")


def select_video():
    global video_path
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi")])
    if video_path:
        play_video()

def select_camera():
    global video_path
    video_path = 0
    play_video()
    
def stop_video():
    global cap
    if cap is not None and cap.isOpened():
        cap.release()
        clear_video_label()
        
def clear_video_label():
    # Clear the video label by setting it to a blank image
    blank_image = Image.new('RGB', (640, 480), color='darkgray') 
    blank_photo = ImageTk.PhotoImage(blank_image)
    video_label.configure(image=blank_photo)
    video_label.image = blank_photo
    video_label.update()


In [7]:
def create_window():
    try:
        # Create Tkinter GUI
        root = tk.Tk()
        root.title("Object Detection Video Application with YOLO")

        # Get the screen width and height
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()

        # Define margin size
        margin = 50
        # Set the geometry with margin
        root.geometry(f"{screen_width - 2 * margin}x{screen_height - 2 * margin}+{margin}+{margin}")

        # Center the window on the screen
        center_window(root)
        
        root.resizable(False, False)

        

        return root
    except tk.TclError as e:
        print(f"Error creating Tkinter window: {e}")
        return None

In [8]:
def center_window(window):
    window.update_idletasks()
    
    width = window.winfo_width()
    height = window.winfo_height()
    
    x_offset = (window.winfo_screenwidth() - width) // 2
    y_offset = (window.winfo_screenheight() - height) // 2
    
    window.geometry(f"{width}x{height}+{x_offset}+{y_offset}")


In [9]:
def create_gui():
    global video_label, root

    if not root.winfo_exists():
        return

    try:
        style = ttk.Style()
        style.theme_use("vista")
        style.configure("TButton", padding=(20, 10), font=('Arial', 12))

        video_frame = tk.Frame(root)
        video_frame.pack(fill="both", expand=True, padx=50, pady=50)

        video_label = tk.Label(video_frame, bg="darkgray", borderwidth=2, relief="groove")
        video_label.pack(fill="both", expand=True)

        button_frame = tk.Frame(root, pady=20)
        button_frame.pack()

        select_video_button = ttk.Button(button_frame, text="Select Video", command=select_video)
        select_video_button.pack(side="left", padx=(0, 10), pady=5)

        select_camera_button = ttk.Button(button_frame, text="Select Camera", command=select_camera)
        select_camera_button.pack(side="right", padx=(10, 0), pady=5)

        stop_button = ttk.Button(button_frame, text="Stop", command=stop_video)
        stop_button.pack(side="right", pady=5)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")


In [10]:
# Main function
def main():
    global net, output_layers, classes, root

    # Load YOLO model and COCO class labels
    net, output_layers = load_yolo()
    classes = load_coco_names()

    # Check if YOLO model and COCO class labels are loaded successfully
    if net is None or output_layers is None or not classes:
        print("Failed to load YOLO model or COCO class labels. Exiting...")
        return

    print("YOLO model and COCO class labels loaded successfully.")

    # Create the Tkinter window
    root = create_window()
    
    # Check if window creation was successful
    if root is None:
        print("Failed to create Tkinter window. Exiting...")
        return

    # Initialize the GUI
    create_gui()

    # Run the Tkinter event loop
    root.mainloop()

In [11]:
main()

YOLO model and COCO class labels loaded successfully.
